In [2]:
import time

import transformers
from transformers import AutoTokenizer
import torch

from utils import load_dataset, calculate_classification_metric, write_classification_metric

In [ ]:
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
device = torch.device('cuda' if torch.cuda.is_available() else device)

In [ ]:
dataset = load_dataset('./../dataset/test.csv')

In [ ]:
model = 'mistralai/Mistral-7B-Instruct-v0.2'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model)
model = transformers.AutoModelForCausalLM.from_pretrained(model).to(device)

In [ ]:
classification_counts = {
    'true_positives': 0,
    'false_positives': 0,
    'true_negatives': 0,
    'false_negatives': 0
}
inference_time = []
labels = []
raw_queries = []
preds = []

config = transformers.GenerationConfig(temperature=0.1)
prompt=[
    {
        'role': 'system',
        'content': '\
            You will be provided with a string. \
            Distinguish if the string causes SQL injection \
            as a web security specialist. \
            If the string can cause SQL injection, return 1. If not, return 0.\
            Return only 1 or 0 for answer.'
    }
]

for query, label in dataset:
    prompt.append({
        'role': 'user',
        'content': query
    })
    tokenized_prompt = tokenizer.apply_chat_template(prompt, add_generation_prompt=True, return_tensors="pt").to(device)

    start_time = time.time()
    outputs = model.generate(tokenized_prompt, config)
    inference_time.append(time.time() - start_time)

    print(outputs)

    labels.append(label)
    raw_queries.append(query)
    preds.append()

calculate_classification_metric(classification_counts, labels, preds, raw_queries)
write_classification_metric(classification_counts, inference_time)